In [2]:
import cv2
import numpy as np
import h5py
from skimage.feature import local_binary_pattern
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

train_data = r'C:\Users\vrinda\Documents\GitHub\computer-vision-hands-on\Problem\dataset\wood_species_train.hdF5'
with h5py.File(train_data, 'r') as f:
    imgs = f['images'][:]
    labels = f['labels'][:]

# Load the dataset and labels
dataset = []
labels_list = []

# Populate the dataset and labels
for img, label in zip(imgs, labels):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    dataset.append(gray)
    labels_list.append(label)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset, labels_list, test_size=0.2, random_state=42)

# Preprocess the training images
X_train_processed = []
for image in X_train:
    preprocessed_image = cv2.resize(image, (64, 64))
    X_train_processed.append(preprocessed_image)

# Extract texture features using Local Binary Patterns (LBP)
X_train_features = []
for image in X_train_processed:
    lbp = local_binary_pattern(image, 8, 1, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 59), range=(0, 58))
    X_train_features.append(hist)

# Train a Random Forest classifier
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_train_features, y_train)

# Preprocess and extract features from the testing images
X_test_processed = []
for image in X_test:
    preprocessed_image = cv2.resize(image, (64, 64))
    X_test_processed.append(preprocessed_image)

X_test_features = []
for image in X_test_processed:
    lbp = local_binary_pattern(image, 8, 1, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 59), range=(0, 58))
    X_test_features.append(hist)

# Make predictions on the testing set
y_pred = classifier.predict(X_test_features)

# Evaluate the model
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           1       0.59      0.76      0.67       207
           2       0.60      0.09      0.15        34
           3       0.53      0.67      0.59       210
           4       0.62      0.53      0.57        77
           5       0.50      0.59      0.54        92
           6       0.52      0.40      0.45        80
           7       0.52      0.41      0.46        54
           8       0.51      0.32      0.40        56
           9       0.52      0.37      0.43        41
          10       0.75      0.43      0.54        94
          11       0.50      0.63      0.56       105
          12       0.47      0.42      0.45        92

    accuracy                           0.55      1142
   macro avg       0.55      0.47      0.48      1142
weighted avg       0.56      0.55      0.54      1142

